In [1]:
using Pkg
Pkg.add("Interpolations")
Pkg.add("Optim")

    Updating registry at `C:\Users\bayle\.julia\registries\General.toml`


   Resolving package versions...

   Installed StaticArraysCore ─ v1.4.2
   Installed Ratios ─────────── v0.4.5


   Installed OffsetArrays ───── v1.12.10
   Installed StaticArrays ───── v1.6.3
   Installed ChainRulesCore ─── v1.16.0


   Installed Interpolations ─── v0.14.7


...

  ✓ ForwardDiff


  ✓ NLSolversBase


  ✓ ForwardDiff → ForwardDiffStaticArraysExt


  ✓ LineSearches


  ✓ Optim
  19 dependencies successfully precompiled in 23 seconds. 160 already precompiled.


In [172]:
using Pkg
Pkg.add("PyPlot")
Pkg.add("LaTeXStrings")

   Resolving package versions...


  No Changes to `C:\Users\bayle\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\bayle\.julia\environments\v1.9\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\bayle\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\bayle\.julia\environments\v1.9\Manifest.toml`

In [4]:
using Interpolations, Optim

In [120]:
"""
A function that provides linear interpolation with constant extrapolation
outside the bounds.

    x : a linear grid of interpolation points
    fx : the values of the function on the grid points

"""
function lin_interp(x, fx)
    return linear_interpolation(x, fx,
    extrapolation_bc = Interpolations.Flat())
    
end

lin_interp

In [121]:
"""
Create an instance of the model, stored as a namedtuple.

"""
function create_opt_savings_model(; β=0.9, #Discount factor
                                    γ=2.0, #CRRA utility parameter
                                    R=1.2, #Gross rate of return
                                    w_size=200, #Grid size for wealth
                                    w_max=10) #maximum wealth
    w_grid = LinRange(1e-8, w_max, w_size) #Create the wealth grid
    u(c)=c^(1-γ)/(1-γ)
    v_init = u.(w_grid) #creating a vector of values for each point in the wealth grid
    return (; β, u, R, w_grid, v_init)
end


create_opt_savings_model

In [122]:
"""
The Bellman operator

    (Tv)(w) = min_{0 ≤ c ≤ w} { u(c) + β v(R(w - c))} Should be max and not min?

"""
function T(v, model)
    (; β, u, R, w_grid) = model #let this define our model

    v_new = similar(v) #create a new vector v_new of exactly the same type as v
    v = lin_interp(w_grid, v)  

    for (i, w) in enumerate(w_grid)
        result = maximize(c -> u(c) + β * v(R * (w-c)), 0.0, w) #choose the value of c which maximises the statement for each point in the wealth grid
        v_new[i]=Optim.maximum(result) 
    end

    return v_new
end


T

In [140]:
" Get a v-greedy policy "
function get_greedy(v, model)
    (; β, u, R, w_grid) = model
    σ = similar(v) #again, create a new vector the same as v. This will be our policy function
    v = lin_interp(w_grid, v)

    for (i,w) in enumerate(w_grid)
        result = maximize(c -> u(c) + β * v(R * (w-c)), 0.0, w)
        σ[i]=Optim.maximizer(result) #Notice the difference to the above. The policy function selects which value of c was the maximiser.
        
    end

    return σ
end

get_greedy

In [124]:
" Value Function Iteration"
function vfi(model;
                tolerance=1e-6,
                max_iter=10_000,
                print_step=100)
    
    k = 0
    error = tolerance + 1 #initial error, we just need this so we can start the loop
    v=model.v_init 

    while (error > tolerance) & (k <= max_iter) 
        v_new = T(v, model)
        error = maximum(abs.(v_new - v)) #supnorm. Error is the maximum pairwise distance
        if k % print_step == 0
            println("Completed iteration $k with error $error.")
        end
        v = v_new
        k += 1
    end

    if error <= tolerance
        println("Terminated successfully in $k iterations")
    else
        println("Warning: hit iteration bound.")    
    end

    σ= get_greedy(v, model) #Given wealth w today, what value of consumption do I choose?
    return σ, v
    
end

vfi

In [147]:
"""
The policy operator

    (T_σ v)(w) =  u(σ(w)) + β v[R(w - σ(w))]

"""
function T_σ(v, σ, model)
    (; β, u, R, w_grid) = model
    # Add your code here
    v_new= similar(v)
    v=lin_interp(w_grid,v)

    for (i, w) in enumerate(w_grid)
        result = maximize(c->u(c)+β*v(R*(w-c)), 0.0, w)
        σ[i]=Optim.maximum(result) #maximum or maximiser?
    end

    return σ
    

end

T_σ

In [141]:
"Approximate lifetime value of policy σ."
function get_value(v_init, σ, m, model)
    v = v_init
    k=0
    while k<m
        v = T_σ(v, σ, model)
        k+=1
    end
    return v
    # Compute and return T_σ^m v_init
    
end

get_value

In [164]:
"Optimistic policy iteration routine."
function opi(model; 
              tolerance=1e-6, 
              max_iter=1_000,
              m=2000, #20
              print_step=10)
    v = model.v_init
    # Put your code here
    k = 0
    error = tolerance + 1

    while (error > tolerance) & (k<= max_iter)
        σ_new=get_greedy(v, model)
        v_new=get_value(v, σ_new, m, model)
        #v_new = T_σ(v, σ, model)
        error=maximum(abs.(v_new - v))
        if k % print_step == 0
            println("Completed iteration $k with error $error.")
        end
        v=v_new
        k+=1
    end

    if error <= tolerance
        println("Terminated successfully in $k iterations.")
    else
        println("Warning: hit iteration bound.")
    end

    σ= get_greedy(v, model)
    return σ, v
end

opi

In [173]:
using PyPlot
using LaTeXStrings
fontsize=12

12

In [162]:
model = create_opt_savings_model()
(; β, u, R, w_grid, v_init) = model;
println("Solving via VFI. \n\n")
@timev σ_star, v_star = vfi(model);

Solving via VFI. 




Completed iteration 0 with error 9.000000634421515e7.
Completed iteration 100 with error 2390.5260684490204.
Completed iteration 200 with error 0.06349575519561768.
Completed iteration 300 with error 1.6689300537109375e-6.
Terminated successfully in 306 iterations
  0.148000 seconds (195.43 k allocations: 16.374 MiB, 65.73% compilation time)
elapsed time (ns):  147999900
gc time (ns):       0
bytes allocated:    17169730
pool allocs:        195325
non-pool GC allocs: 108
minor collections:  0
full collections:   0


In [176]:
plot(σ_star, v_star)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x0000023A8C4B7CD0>

In [148]:
#model = create_opt_savings_model()
#(; β, u, R, w_grid, v_init) = model;
#σ=get_greedy(v_init, model)
#v=get_value(v_init, σ, 20, model)

200-element Vector{Float64}:
    -8.905810665975548e8
 -1051.2566445255231
  -520.6074761577372
  -344.07368479585824
  -256.3488263154071
  -204.15797454939823
  -169.60664909493346
  -145.0483871903849
  -126.69561448776179
  -112.46191731076158
     ⋮
    -5.3039446414704265
    -5.280657722036804
    -5.25777229758961
    -5.235710990623158
    -5.2140763479544905
    -5.192858086430046
    -5.1725929463725695
    -5.152787887396123
    -5.1341350381266375

In [166]:
println("\n\nSolving via HPI.\n\n")
@timev σ_star_opi, v_star_hpi = opi(model); #Run time for m=20 is 0.05s. Run time for m=2000 is 0.72s. Run time for m=20000 is 6.45s. Just change the value of m in OPI above.



Solving via HPI.




Completed iteration 0 with error 9.000000634421512e8.


Terminated successfully in 2 iterations.
  0.690121 seconds (1.61 M allocations: 135.982 MiB, 2.06% gc time)
elapsed time (ns):  690121300
gc time (ns):       14221400
bytes allocated:    142587432
pool allocs:        1609357
non-pool GC allocs: 0
minor collections:  1
full collections:   0


In [177]:
plot(σ_star_opi, σ_star, show=true)

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\bayle\.julia\packages\PyCall\twYvK\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'AttributeError'>
AttributeError("Line2D.set() got an unexpected keyword argument 'show'")
  File "C:\Users\bayle\.julia\conda\3\x86_64\lib\site-packages\matplotlib\pyplot.py", line 2812, in plot
    return gca().plot(
  File "C:\Users\bayle\.julia\conda\3\x86_64\lib\site-packages\matplotlib\axes\_axes.py", line 1688, in plot
    lines = [*self._get_lines(*args, data=data, **kwargs)]
  File "C:\Users\bayle\.julia\conda\3\x86_64\lib\site-packages\matplotlib\axes\_base.py", line 311, in __call__
    yield from self._plot_args(
  File "C:\Users\bayle\.julia\conda\3\x86_64\lib\site-packages\matplotlib\axes\_base.py", line 544, in _plot_args
    return [l[0] for l in result]
  File "C:\Users\bayle\.julia\conda\3\x86_64\lib\site-packages\matplotlib\axes\_base.py", line 544, in <listcomp>
    re

In [175]:
plot([v_star_hpi v_star])

2-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x0000023A8C4B7760>
 PyObject <matplotlib.lines.Line2D object at 0x0000023A8C4B7790>